# MCBU CH4 GIS - EDF PermianMap CarbonMapper plumes

## Import libraries

In [1]:
import os
import datetime
import json
import shutil

import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
from arcgis.features import GeoAccessor
import contextily as cx

#https://towardsdatascience.com/how-to-set-up-anaconda-and-jupyter-notebook-the-right-way-de3b7623ea4a

## Load the plumes data from AGOL into a gdf

In [2]:
# Connect to AGOL
agol = GIS()

# Get the plumes data using the itemid
# https://www.arcgis.com/home/item.html?id=10a98c998f264570939b507d324e8330#overview
plumes_data = agol.content.get('10a98c998f264570939b507d324e8330').layers[0]

# .query() returns a FeatureSet
fset = plumes_data.query()

# get a GeoJSON string representation of the FeatureSet
gjson_string = fset.to_geojson

# read GeoJSON string into a dict
gjson_dict = json.loads(gjson_string)

# Read the dict into a gdf
plumes_gdf = gpd.GeoDataFrame.from_features(gjson_dict['features'])

# Set the CRS
plumes_gdf = plumes_gdf.set_crs(epsg=3857)
#plumes_gdf

In [4]:
plumes_gdf.head()

,geometry,OBJECTID,source_id,plume_lat,plume_lon,plume_candidate_id,date_of_detection,time_of_detection,source_type,source_type_label,qplume,sigma_qplume,active_flare,inactive_flare,Operator_Alias,County,State
0,POINT (-11594403.640 3776182.850),9827,F21P_0001,32.098010,-104.15430,GAO20211003t155009p0000-A,1633219200000,15:50:09,well,Production,237.703647,70.335432,None,None,CHEVRON USA,Eddy,New Mexico
1,POINT (-11590221.367 3782859.353),9828,F21P_0002,32.148804,-104.11673,GAO20211003t155009p0000-B,1633219200000,15:50:09,tank,Production,488.323421,223.227806,None,None,OCCIDENTAL PETROLEUM COMPANY,Eddy,New Mexico
2,POINT (-11585571.552 3785645.330),9829,F21P_0003,32.169991,-104.07496,GAO20211003t155009p0000-C,1633219200000,15:50:09,compressor,Gathering & Boosting,582.313511,172.122195,None,None,SENDERO MIDSTREAM,Eddy,New Mexico
3,POINT (-11575563.929 3793028.206),9830,F21P_0004,32.226113,-103.98506,GAO20211003t155009p0000-D,1633219200000,15:50:09,compressor,Gathering & Boosting,720.878562,199.999978,None,None,DCP MIDSTREAM PARTNERS,Eddy,New Mexico
4,POINT (-11574385.056 3792413.955),9831,F21P_0005,32.221445,-103.97447,GAO20211003t155009p0000-E,1633219200000,15:50:09,compressor,Gathering & Boosting,666.558008,262.897112,None,None,ENTERPRISE PRODUCTS,Eddy,New Mexico


## Connect to Chevron MapHub

In [91]:
gis = GIS("https://maphub-amer.tst.azure.chevron.com/portal", client_id='bb6vn14P957pN6bV')
print("Logged in as: " + gis.properties.user.username)

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://maphub-amer.tst.azure.chevron.com/portal/sharing/rest//oauth2/authorize?response_type=code&client_id=bb6vn14P957pN6bV&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=wFQSabLwz2fJNHAgXP7fDRvkOrIU5v
Enter code obtained on signing in using SAML: ········


C:\Users\ekqo\Miniconda3\envs\arcgis\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maphub-amer.tst.azure.chevron.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Logged in as: ekqo


## Load the MCBU field outlines into a gdf

In [96]:
# Title for fields data
title = 'COMM_MCBU_OPS_FOT_OUTLINE'

# Search for the plumes data by title
fields_id = gis.content.search(query='title:' + title,
                               item_type='Feature Layer',
                               max_items=35)

# Get the field outlines data using the itemid
fields_data = gis.content.get(fields_id[0].itemid).layers[0]

# .query() returns a FeatureSet
fset = fields_data.query()

# get a GeoJSON string representation of the FeatureSet
gjson_string = fset.to_geojson

# read GeoJSON string into a dict
gjson_dict = json.loads(gjson_string)

# Read the dict into a gdf
fields_gdf = gpd.GeoDataFrame.from_features(gjson_dict['features'])

# Set the CRS
fields_gdf = fields_gdf.set_crs(epsg=4267)
fields_gdf

,geometry,OBJECTID,AREA_NAME,FIELD_OPERATION_TEAM_NAME,PRODUCTION_TEAM_NAME,FIELD_CODE,FIELD_NAME,MCBU_GIS_DATA_SOURCE_TYPE,MCBU_GIS_SOURCE_NAME,MCBU_GIS_DATA_RECEIVED_DATE,MCBU_GIS_DATA_UPLOADED_DATE,MCBU_GIS_DATA_MODIFIED_DATE,MCBU_GIS_DATA_MODIFIED_BY,MCBU_GIS_DATA_REMARK,CURRENT_INDICATOR_LF,MCBU_GIS_OPS_CONTACT,SHAPE.STArea(),SHAPE.STLength()
0,"POLYGON ((-103.32889 31.23542, -103.34337 31.2...",2021,CENTRAL AREA,TEXAS,FT STOCKTON,UCS-QV2H,WOLFBONE (TX) HZ (REEVES),EXCEL,MCBU Production Reporting (MPR),1631664000000,1632096000000,1632096000000,Janice Fisher (KCVL),None,Y,Jennifer Pinnick,0.002304,0.492089
1,"POLYGON ((-102.14206 31.93012, -102.14206 31.9...",2022,MIDLAND BASIN,ODESSA,ODESSA CENTRAL,UCS-PF8H,BRYANT -G- HORIZONTALS,EXCEL,MCBU Production Reporting (MPR),1631664000000,1632096000000,1632096000000,Janice Fisher (KCVL),None,Y,Matt Anderson,0.003142,0.578932
2,"POLYGON ((-101.95390 31.62620, -101.96838 31.6...",2023,MIDLAND BASIN,ODESSA,ODESSA CENTRAL,UCS-QP5,SPRAYBERRY TREND,EXCEL,MCBU Production Reporting (MPR),1631664000000,1632096000000,1632096000000,Janice Fisher (KCVL),None,Y,Matt Anderson,0.001676,0.231572
3,"POLYGON ((-102.01180 31.84329, -102.01180 31.8...",2024,MIDLAND BASIN,ODESSA,ODESSA EAST,UCS-QP5H,SPRAYBERRY TREND - CMC HZ,EXCEL,MCBU Production Reporting (MPR),1631664000000,1632096000000,1632096000000,Janice Fisher (KCVL),None,Y,Matt Anderson,0.002304,0.405253
4,"POLYGON ((-101.66443 32.17617, -101.64996 32.1...",2025,MIDLAND BASIN,ODESSA,ODESSA EAST,UCS-QX1,SPRAYBERRY TREND VIOLET,EXCEL,MCBU Production Reporting (MPR),1631664000000,1632096000000,1632096000000,Janice Fisher (KCVL),None,Y,Matt Anderson,0.000628,0.144732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,"POLYGON ((-103.31397 32.10367, -103.32844 32.1...",2168,CENTRAL AREA,PERMIAN CO2 WATERFLOODS,HOBBS,UCS-LG9,WILDCAT,EXCEL,MCBU Production Reporting (MPR),1631664000000,1635724800000,1635724800000,Janice Fisher (KCVL),None,Y,Jennifer Pinnick,0.000628,0.173679
148,"POLYGON ((-103.79158 32.72602, -103.79158 32.7...",2169,CENTRAL AREA,PERMIAN CO2 WATERFLOODS,HOBBS,UCS-RK1,YOUNG,EXCEL,MCBU Production Reporting (MPR),1631664000000,1635724800000,1635724800000,Janice Fisher (KCVL),None,Y,Jennifer Pinnick,0.000419,0.115786
149,"POLYGON ((-103.79158 32.74049, -103.80605 32.7...",2170,CENTRAL AREA,PERMIAN CO2 WATERFLOODS,HOBBS,UCS-EB7,YOUNG NORTH,EXCEL,MCBU Production Reporting (MPR),1631664000000,1635724800000,1635724800000,Janice Fisher (KCVL),None,Y,Jennifer Pinnick,0.000209,0.057893
150,"POLYGON ((-103.76264 32.69707, -103.77711 32.6...",2171,CENTRAL AREA,PERMIAN CO2 WATERFLOODS,HOBBS,UCS-KP5,YOUNG SOUTH,EXCEL,MCBU Production Reporting (MPR),1631664000000,1635724800000,1635724800000,Janice Fisher (KCVL),None,Y,Jennifer Pinnick,0.000209,0.057893


## Reproject the plumes data

In [95]:
# Reproject the plumes
plumes_gdf = plumes_gdf.to_crs(epsg=4267)
plumes_gdf

,geometry,OBJECTID,source_id,plume_lat,plume_lon,plume_candidate_id,date_of_detection,time_of_detection,source_type,source_type_label,ipcc,qplume,sigma_qplume,active_flare,inactive_flare,Operator_Alias,imageurl,County,State
0,POINT (-104.15381 32.09789),9827,F21P_0001,32.098010,-104.154300,GAO20211003t155009p0000-A,1633219200000,15:50:09,well,Production,1B2 Oil & Natural Gas,237.703647,70.335432,None,None,CHEVRON USA,https://edf-permian-data.s3.amazonaws.com/imag...,Eddy,New Mexico
1,POINT (-104.11624 32.14868),9828,F21P_0002,32.148804,-104.116730,GAO20211003t155009p0000-B,1633219200000,15:50:09,tank,Production,1B2 Oil & Natural Gas,488.323421,223.227806,None,None,OCCIDENTAL PETROLEUM COMPANY,https://edf-permian-data.s3.amazonaws.com/imag...,Eddy,New Mexico
2,POINT (-104.07447 32.16987),9829,F21P_0003,32.169991,-104.074960,GAO20211003t155009p0000-C,1633219200000,15:50:09,compressor,Gathering & Boosting,1B2 Oil & Natural Gas,582.313511,172.122195,None,None,SENDERO MIDSTREAM,https://edf-permian-data.s3.amazonaws.com/imag...,Eddy,New Mexico
3,POINT (-103.98457 32.22599),9830,F21P_0004,32.226113,-103.985060,GAO20211003t155009p0000-D,1633219200000,15:50:09,compressor,Gathering & Boosting,1B2 Oil & Natural Gas,720.878562,199.999978,None,None,DCP MIDSTREAM PARTNERS,https://edf-permian-data.s3.amazonaws.com/imag...,Eddy,New Mexico
4,POINT (-103.97398 32.22132),9831,F21P_0005,32.221445,-103.974470,GAO20211003t155009p0000-E,1633219200000,15:50:09,compressor,Gathering & Boosting,1B2 Oil & Natural Gas,666.558008,262.897112,None,None,ENTERPRISE PRODUCTS,https://edf-permian-data.s3.amazonaws.com/imag...,Eddy,New Mexico
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
768,POINT (-103.40342 31.74680),10595,F21P_0457,31.746928,-103.403874,GAO20211017t181626p0000-A,1634428800000,18:16:26,unknown,None,1B2 Oil & Natural Gas,177.910257,98.995010,None,None,UNKNOWN,https://edf-permian-data.s3.amazonaws.com/imag...,Loving,Texas
769,POINT (-103.53397 31.67499),10596,F21P_0200,31.675121,-103.534422,GAO20211017t181626p0000-B,1634428800000,18:16:26,tank,Production,1B2 Oil & Natural Gas,222.816707,47.896354,None,None,DIAMONDBACK ENERGY INC,https://edf-permian-data.s3.amazonaws.com/imag...,Loving,Texas
770,POINT (-103.44691 31.72017),10597,F21P_0340,31.720298,-103.447368,GAO20211017t181626p0000-C,1634428800000,18:16:26,well,Production,1B2 Oil & Natural Gas,250.746178,60.359406,None,None,OCCIDENTAL PETROLEUM COMPANY,https://edf-permian-data.s3.amazonaws.com/imag...,Loving,Texas
771,POINT (-103.51206 31.75744),10598,F21P_0194,31.757567,-103.512514,GAO20211017t183345p0000-A,1634428800000,18:33:45,processing,Processing,1B2 Oil & Natural Gas,217.586591,75.081864,None,None,ENLINK,https://edf-permian-data.s3.amazonaws.com/imag...,Loving,Texas


## Clip plumes to field outlines 

In [98]:
# Use spatial join to clip points to field outlines
plumes_gdf_clipped = gpd.sjoin(plumes_gdf, fields_gdf, how='inner')

# Rename lat/long
plumes_gdf_clipped.rename(columns={'plume_lat': 'Lat', 'plume_lon': 'Long'}, inplace=True)
plumes_gdf_clipped

,geometry,OBJECTID_left,source_id,Lat,Long,plume_candidate_id,date_of_detection,time_of_detection,source_type,source_type_label,...,MCBU_GIS_SOURCE_NAME,MCBU_GIS_DATA_RECEIVED_DATE,MCBU_GIS_DATA_UPLOADED_DATE,MCBU_GIS_DATA_MODIFIED_DATE,MCBU_GIS_DATA_MODIFIED_BY,MCBU_GIS_DATA_REMARK,CURRENT_INDICATOR_LF,MCBU_GIS_OPS_CONTACT,SHAPE.STArea(),SHAPE.STLength()
2,POINT (-104.07447 32.16987),9829,F21P_0003,32.169991,-104.07496,GAO20211003t155009p0000-C,1633219200000,15:50:09,compressor,Gathering & Boosting,...,MCBU Production Reporting (MPR),1631664000000,1632096000000,1632096000000,Janice Fisher (KCVL),None,Y,Darrell Carriger,0.000209,0.057894
437,POINT (-104.07452 32.16997),10264,F21P_0003,32.170096,-104.07501,GAO20211009t153114p0000-D,1633737600000,15:31:14,compressor,Gathering & Boosting,...,MCBU Production Reporting (MPR),1631664000000,1632096000000,1632096000000,Janice Fisher (KCVL),None,Y,Darrell Carriger,0.000209,0.057894
590,POINT (-104.07417 32.17226),10417,F21P_0372,32.172384,-104.07466,GAO20211013t182617p0000-A,1634083200000,18:26:17,compressor,Gathering & Boosting,...,MCBU Production Reporting (MPR),1631664000000,1632096000000,1632096000000,Janice Fisher (KCVL),None,Y,Darrell Carriger,0.000209,0.057894
617,POINT (-104.07458 32.16999),10444,F21P_0003,32.170113,-104.07507,GAO20211014t152820p0000-A,1634169600000,15:28:20,compressor,Gathering & Boosting,...,MCBU Production Reporting (MPR),1631664000000,1632096000000,1632096000000,Janice Fisher (KCVL),None,Y,Darrell Carriger,0.000209,0.057894
159,POINT (-103.60457 32.02314),9986,F21P_0141,32.023260,-103.60504,GAO20211005t151211p0000-C,1633392000000,15:12:11,well,Production,...,MCBU Production Reporting (MPR),1631664000000,1632096000000,1632096000000,Janice Fisher (KCVL),None,Y,Darrell Carriger,0.001885,0.289470
368,POINT (-103.72130 32.25196),10195,F21P_0277,32.252086,-103.72178,GAO20211007t174844p0000-B,1633564800000,17:48:44,tank,Production,...,MCBU Production Reporting (MPR),1631664000000,1632096000000,1632096000000,Janice Fisher (KCVL),None,Y,Darrell Carriger,0.000628,0.115787
378,POINT (-102.26186 31.63372),10205,F21P_0283,31.633856,-102.26228,GAO20211008t150536p0000-A,1633651200000,15:05:36,well,Production,...,MCBU Production Reporting (MPR),1631664000000,1632096000000,1632096000000,Janice Fisher (KCVL),None,Y,Matt Anderson,0.005027,0.376302
762,POINT (-102.29678 31.66160),10589,F21P_0452,31.661732,-102.29720,GAO20211016t190642p0000-A,1634342400000,19:06:42,compressor,Gathering & Boosting,...,MCBU Production Reporting (MPR),1631664000000,1632096000000,1632096000000,Janice Fisher (KCVL),None,Y,Matt Anderson,0.005027,0.376302
460,POINT (-103.65805 32.24128),10287,F21P_0310,32.241404,-103.65853,GAO20211009t172201p0000-A,1633737600000,17:22:01,unknown,None,...,MCBU Production Reporting (MPR),1631664000000,1632096000000,1632096000000,Janice Fisher (KCVL),None,Y,Darrell Carriger,0.000419,0.086840
465,POINT (-103.65812 32.24131),10292,F21P_0310,32.241434,-103.65860,GAO20211009t173315p0000-B,1633737600000,17:33:15,unknown,None,...,MCBU Production Reporting (MPR),1631664000000,1632096000000,1632096000000,Janice Fisher (KCVL),None,Y,Darrell Carriger,0.000419,0.086840


## Publish feature layer to Chevron MapHub

In [99]:
# Convert the GDF to PD
plumes_pd_clipped = pd.DataFrame(plumes_gdf_clipped.drop(columns='geometry'))

# Import the dataframe
plumes_pd = gis.content.import_data(plumes_pd_clipped)
plumes_pd

<FeatureCollection>

In [100]:
# Create JSON for df
plumes_pd_dict = dict(plumes_pd.properties)
plumes_json = json.dumps({"featureCollection": {"layers": [plumes_pd_dict]}})

In [101]:
# Add item properties
plumes_item_properties = {'title': 'EDF/PermianMap - Fall 2021 Plumes detected by GAO/Carbon Mapper for MCBU fields',
                          'description': 'This dataset includes the detections of individual CH4 plumes during the Fall 2021 deployment of the GAO aircraft in the Permian Basin by CarbonMapper. The data has been clipped to MCBU field outlines.',
                          'tags': 'CTC-HSE, MCBU, CH4',
                          'text': plumes_json,
                          'type': 'Feature Collection'}

# Create feature layer
plumes_item = gis.content.add(plumes_item_properties)
plumes_item

<Item title:"EDF/PermianMap - Fall 2021 Plumes detected by GAO/Carbon Mapper for MCBU fields" type:Feature Collection owner:ekqo>

In [106]:
# Share the item with the MCBU_CH4 group
plumes_item.share(groups='0197787ca28344859cba1b89638f185c')

{'results': [{'itemId': '1a846cd08b0a422db7abc72d7e32637e',
   'success': True,
   'notSharedWith': []}]}